In this script, the landuse area (i.e. exact polygon) is added to the mines included in the production dataset. 

In [53]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium

## Read Data

In [2]:
facilities = gpd.read_file("data/production/facilities.gpkg")
polygons = gpd.read_file("data/polygons/polygons.gpkg")

## Preprocessing of facilities

In [3]:
# only keep facilities, not sub-sites
facilities = facilities[facilities["sub_site_name"].isnull()]

# only keep facilities that have non-empty geometry
facilities = facilities[~facilities.is_empty]

## View the data

In [4]:
# display(facilities)
# display(polygons)
# facilities.explore()
# polygons.explore()

In [5]:
### TEST: what happens if we intersect a multipoint with polygons? 
test_facilities = facilities[facilities["facility_id"] == "COM00007.00"]
display(test_facilities)
gpd.sjoin(test_facilities, polygons, how='inner', predicate='intersects', lsuffix='left', rsuffix='right')

,facility_id,facility_name,facility_other_names,sub_site_name,sub_site_other_names,facility_type,primary_commodity,commodities_products,facility_equipment,production_start,...,concession_area_sq_km,country,GID_0,GID_1,GID_2,GID_3,GID_4,source_id,comment,geometry
6,COM00007.00,AGA Mineracao,None,None,None,Mine,Gold,Gold,"Underground, Open pit, Heap leaching plant",NaN,...,NaN,Brazil,BRA,BRA.13_1,BRA.13.662_1 ; BRA.13.672_1,BRA.13.662.2_1 ; BRA.13.662.4_1 ; BRA.13.672.1...,None,det_1057,None,"MULTIPOINT (-43.73907 -19.86773, -43.76980 -19..."


,facility_id,facility_name,facility_other_names,sub_site_name,sub_site_other_names,facility_type,primary_commodity,commodities_products,facility_equipment,production_start,...,GID_2,GID_3,GID_4,source_id,comment,geometry,index_right,ISO3_CODE,COUNTRY_NAME,AREA
6,COM00007.00,AGA Mineracao,None,None,None,Mine,Gold,Gold,"Underground, Open pit, Heap leaching plant",NaN,...,BRA.13.662_1 ; BRA.13.672_1,BRA.13.662.2_1 ; BRA.13.662.4_1 ; BRA.13.672.1...,None,det_1057,None,"MULTIPOINT (-43.73907 -19.86773, -43.76980 -19...",3230,BRA,Brazil,0.246638
6,COM00007.00,AGA Mineracao,None,None,None,Mine,Gold,Gold,"Underground, Open pit, Heap leaching plant",NaN,...,BRA.13.662_1 ; BRA.13.672_1,BRA.13.662.2_1 ; BRA.13.662.4_1 ; BRA.13.672.1...,None,det_1057,None,"MULTIPOINT (-43.73907 -19.86773, -43.76980 -19...",3263,BRA,Brazil,0.172577
6,COM00007.00,AGA Mineracao,None,None,None,Mine,Gold,Gold,"Underground, Open pit, Heap leaching plant",NaN,...,BRA.13.662_1 ; BRA.13.672_1,BRA.13.662.2_1 ; BRA.13.662.4_1 ; BRA.13.672.1...,None,det_1057,None,"MULTIPOINT (-43.73907 -19.86773, -43.76980 -19...",3302,BRA,Brazil,0.830338


Conclusion of the test: a spatial join of a multipoint with polygons returns all the intersections of the multipoint and the poylgons. 

## Join the data
We want two dataframes:
- One containing all mines included in the production data, including the mine coordinates, and the (total of all multipoints) area of the intersecting polygon
- Another one with just the polygons intersecting with point coordinates, that we can then add as a layer to the geopackage. 

For now, we match polygons that intersect with mine points only. Then, we check how many intersections we get.
However, later we want to also test to intersect a radius around the points with the polygons. 

### 1. Dataframe: all mines with area

In [58]:
# produce the intersection
df = gpd.sjoin(facilities, polygons, how='left', predicate='intersects', lsuffix='left', rsuffix='right')
# display(df)

# for the mines that are represented with multipoints, we have to add up the mining area. 
area = df.groupby("facility_id", as_index = False).sum("AREA").loc[:,["facility_id", "AREA"]]

facilities_merged = pd.merge(facilities, area, how = "left", on="facility_id")
facilities_merged["AREA"].replace(0, np.nan, inplace = True)

### 2. Dataframe: all polygons intersecting with mines

In [65]:
polygons_merged = gpd.sjoin(polygons, facilities, how = "inner", predicate="intersects").iloc[:,range(4)]
print(polygons_merged.shape)

# only keep unique polygons, we dont want duplicates
polygons_merged = polygons_merged.drop_duplicates("geometry")

print(polygons_merged.shape)
display(polygons_merged)


(1047, 4)
(991, 4)


,ISO3_CODE,COUNTRY_NAME,AREA,geometry
10,NZL,New Zealand,0.617854,"POLYGON ((175.83980 -37.38440, 175.83920 -37.3..."
81,NZL,New Zealand,18.070775,"POLYGON ((170.44460 -45.39300, 170.45420 -45.3..."
99,NCL,New Caledonia,17.480816,"POLYGON ((166.90410 -22.31050, 166.90190 -22.3..."
243,AUS,Australia,7.054856,"POLYGON ((146.84040 -41.12200, 146.83870 -41.1..."
253,AUS,Australia,5.706840,"POLYGON ((145.59420 -42.06830, 145.59460 -42.0..."
...,...,...,...,...
44531,RUS,Russian Federation,26.788483,"POLYGON ((86.05950 55.66570, 86.06260 55.66570..."
44537,CAN,Canada,60.829266,"POLYGON ((-102.94683 49.13949, -102.94666 49.1..."
44603,CHN,China,6.111370,"POLYGON ((125.78161 50.13985, 125.78030 50.140..."
44650,RUS,Russian Federation,8.497812,"POLYGON ((169.56007 66.81152, 169.55612 66.809..."


## Visualize again to validate

In [66]:
m = polygons_merged.explore(
    color = "red",
    name = "polygons"
)

facilities_merged.explore(
    m = m,
    color = "blue", 
    marker_kwds = dict(radius=3, fill=True),
    tooltip="facility_name",
    name = "facilities"
)

folium.LayerControl().add_to(m)
m

## Export files to intermediate

In [69]:
import os

# create the intermediate directory if it does not exist
path = "./intermediate"
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
    print("The new directory is created!")

# write the joined dataframe to intermediate 
df.to_file("intermediate/data_merged.gpkg", layer = "facilities")
df.to_file("intermediate/data_merged.gpkg", layer = "polygons")